In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train_unzip.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Data Cleanup

In [3]:
def cleanup2(x1,new_missing_percent,a):
    s = (len(x1[:,0]),len(x1[0,:]))
    logic_array = np.ones(s)
    for k in range(len(x1[0,:])):

        if(new_missing_percent[k]>0):
            logic_array[:,k] = (x1[:,k]!=-999).astype(int)
    x2= np.multiply(logic_array,x1) #array with zeros instead of -999

    mean = np.mean(x2,0) #vector with mean of each column without -999
    means = np.multiply(1-logic_array,mean) # zeros-array with mean of column instead of missing value
    x3=np.add(x2,means) # array cleaned, each -999 is replaced with mean of column
    return x3

In [35]:
# Remove features with more than 70% missing values and store the output in x1 array
def cleanup(x1):
    missing_percent = np.zeros(len(x1[0,:]))
    new_missing_percent = []
    i = 0
    idx_del = 0
    for i in range(len(x1[0,:])):
        a = ((x1[:,i]) == -999).astype(int)
        c = np.sum(a)/len(a)
        missing_percent[i] = c
        if c>=0.7:
            idx_del = np.append(idx_del,i) #save the index of the deletet features that the same can be removed form the test dat  
        new_missing_percent = np.append(new_missing_percent,c)
    x1 = np.delete(x1,idx_del,1)
    print("Missing values' percentage for each column is: \n",  missing_percent)
    print("New missing values' percentage after 70% threshold: \n",  new_missing_percent)
    print("Intial number of features is \n", len(tX[0,:]))
    print("Number of deleted features is \n", len(tX[0,:])-len(x1[0,:]))
    
    return cleanup2(x1,new_missing_percent,a), idx_del
x3, idx_del = cleanup(tX)

Missing values' percentage for each column is: 
 [0.152456 0.       0.       0.       0.709828 0.709828 0.709828 0.
 0.       0.       0.       0.       0.709828 0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.399652
 0.399652 0.399652 0.709828 0.709828 0.709828 0.      ]
New missing values' percentage after 70% threshold: 
 [0.152456 0.       0.       0.       0.709828 0.709828 0.709828 0.
 0.       0.       0.       0.       0.709828 0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.399652
 0.399652 0.399652 0.709828 0.709828 0.709828 0.      ]
Intial number of features is 
 30
Number of deleted features is 
 8


## Do your thing crazy machine learning thing here :) ...

In [37]:
from implementations import *

In [7]:
"""
# Logistic regression, does not work sofar
initial_w = np.zeros((tx.shape[1], 1))
max_iters = 10
gamma = 0.05
w, mse = logistic_regression(y, tx, initial_w, max_iters, gamma)
"""

'\n# Logistic regression, does not work sofar\ninitial_w = np.zeros((tx.shape[1], 1))\nmax_iters = 10\ngamma = 0.05\nw, mse = logistic_regression(y, tx, initial_w, max_iters, gamma)\n'

In [23]:
len(x3)

TypeError: 'int' object is not subscriptable

In [57]:
### Implementation of pearson coefficent
def claculatePearson(y,x3,limit):
    """
    y:
    x3:
    limit: 
    Returns the indexes of the absolut value of the pearson coefficent which are smaller than the limit
    """
    yandx = np.c_[y,x3]
    pearson = []
    for i in range(len(x3[0,:])):
        corr =  np.corrcoef(x3[:,i ],y)
        pearson = np.append(pearson,corr[0,1])
    idx_pearson = (~(np.abs(pearson[:])>0.1)).astype(int)
    return idx_pearson

In [59]:
x4 = np.delete(x3,claculatePearson(y,x3,0.1),1)

In [70]:
tx = np.c_[np.ones((y.shape[0], 1)), x4] # Add artifical feature to get non zero intercept

In [71]:
w, mse = least_squares(y,tx) #Least square implementation

## Generate predictions and save ouput in csv format for submission:

In [72]:
DATA_TEST_PATH = 'data/test_unzip.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [73]:
tX_test2 = np.delete(tX_test,idx_del,1)

In [74]:
tX_test3 = np.delete(tX_test2,idx_pearson,1)

In [75]:
tX_test4 = np.c_[np.ones((tX_test.shape[0], 1)), tX_test3]

In [76]:
tX_test4.shape

(568238, 21)

In [77]:
OUTPUT_PATH = 'data/prediction.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test4)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)